In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
from datetime import datetime
from loguru import logger
import numpy as np
import spatialdata as sd
from napari_spatialdata import Interactive

from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.processors import build_processor
from multiplex_pipeline.processors.controller import ResourceBuildingController
from multiplex_pipeline.object_quantification.controller import QuantificationController

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [2]:
# load analysis configuration
settings_path = r'C:\BLCA-2_Analysis_todel\analysis_settings_BLCA2_todel.yaml'

settings = load_analysis_settings(settings_path)
settings

{'image_dir': 'R:/CellDive/BLCA-2/BLCA-2_Final',
 'analysis_name': 'BLCA-2_Analysis_todel',
 'local_analysis_dir': 'C:/',
 'remote_analysis_dir': '/ix1/kkedziora/blca_analysis',
 'log_dir': WindowsPath('C:/BLCA-2_Analysis_todel/logs'),
 'detection_image': 'BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('C:/BLCA-2_Analysis_todel/cores.csv'),
 'cores_dir_tif': WindowsPath('C:/BLCA-2_Analysis_todel/temp'),
 'cores_dir_output': WindowsPath('C:/BLCA-2_Analysis_todel/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': 'DAPI',
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  'CD25',
  'CD19',
  'CD27',
  'CCR7',
  'cCASP3'],
 'additional_elements': [{'category': 'image_transfo

### Define the logger

In [3]:
log_file = settings['log_dir'] / f"cores_quantification_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
logger.add(lambda msg: print(msg, end=""))
logger.add(log_file, level="DEBUG", enqueue=True)

2

### Define cores for the analysis

In [4]:
core_dir = Path(settings['analysis_dir']) / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/BLCA-2_Analysis_todel/cores/Core_000.zarr')]

### Setup

In [11]:
# setup quantification controllers
quant_controller_list = [] 
for quant in settings['quant']:

    table_name = quant['name']
    masks_keys = quant['masks']
    # # for test - delete
    # masks_keys = {k: masks_keys[k] for k in ['nucleus', 'cell'] if k in masks_keys}
    connect_to_mask = quant.get('layer_connection', None)

    logger.info(f"Setting up quantification controller for '{table_name}' table with masks {masks_keys} and connection to '{connect_to_mask}' mask")

    controller = QuantificationController(
        table_name=table_name,
        mask_keys=masks_keys,
        connect_to_mask=connect_to_mask,
        overwrite=True,
    )

    quant_controller_list.append(controller) 

2025-10-14 17:09:09.088 | INFO     | __main__:<module>:11 - Setting up quantification controller for 'instanseg_table' table with masks {'nucleus': 'instanseg_nucleus', 'cell': 'instanseg_cell', 'ring': 'ring', 'cyto': 'cytoplasm'} and connection to 'instanseg_cell' mask


### Quantify

In [12]:
for sd_path in path_list[:1]:
    
    # load data
    logger.info(f'Processing {sd_path.name}')
    sdata = sd.read_zarr(sd_path)

    # run quantification
    for controller in quant_controller_list:
        controller.run(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-10-14 17:09:10.348 | INFO     | __main__:<module>:4 - Processing Core_000.zarr


c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


2025-10-14 17:09:10.648 | INFO     | multiplex_pipeline.object_quantification.controller:validate_sdata_as_input:20 - Channels not specified. Quantifying all existing channels (1).
2025-10-14 17:09:10.648 | WARNING  | multiplex_pipeline.object_quantification.controller:prepare_to_overwrite:8 - Table 'instanseg_table' already exists and will be overwritten.
2025-10-14 17:09:10.982 | INFO     | multiplex_pipeline.object_quantification.controller:prepare_to_overwrite:11 - Existing table 'instanseg_table' deleted from sdata.
2025-10-14 17:09:13.223 | INFO     | multiplex_pipeline.object_quantification.controller:run:7 - Prepared masks for quantification.
2025-10-14 17:09:13.226 | INFO     | multiplex_pipeline.object_quantification.controller:build_obs:74 - Quantifying morphology features for mask 'nucleus'
2025-10-14 17:09:26.626 | INFO     | multiplex_pipeline.object_quantification.controller:build_obs:74 - Quantifying morphology features for mask 'cell'
2025-10-14 17:09:42.909 | INFO    

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\spatialdata\models\models.py:1144: UserWarning: Converting `region_key: region` to categorical dtype.
  return convert_region_column_to_categorical(adata)


2025-10-14 17:10:17.630 | INFO     | multiplex_pipeline.object_quantification.controller:run:19 - Quantification complete. Resulting AnnData has 13866 observations and 8 variables.
2025-10-14 17:10:18.698 | SUCCESS  | multiplex_pipeline.object_quantification.controller:run:28 - Quantification complete. Table 'instanseg_table' written to C:\BLCA-2_Analysis_todel\cores\Core_000.zarr


In [10]:
sdata = sd.read_zarr(sd_path)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


In [ ]:
from napari_spatialdata import Interactive

Interactive(sdata)

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-10-10 11:55:27.783 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:55:27.787 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:55:32.836 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:55:32.841 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:55:34.285 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:56:30.828 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:56:30.828 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:56:30.838 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
